In [5]:
import numpy as np
from src.consequential_learning import collect_data, train
from src.feature_map import IdentityFeatureMap

In [6]:
DIM_X = 1
DIM_S = 1
DIM_THETA = DIM_S + DIM_X
T = 200
NUM_ITERATIONS = 32
BATCH_SIZE = 512
NUM_DECISIONS = NUM_ITERATIONS * BATCH_SIZE
LEARNING_RATE = 0.5
FAIRNESS_RATE = 0
COST_FACTOR = 0.55
FRACTION_PROTECTED = 0.3

In [7]:
def benefit_function(x_s, s, sample_theta, policy, gradient):
    ips_weight, phi, log_gradient_denominator = policy.calculate_ips_weights_and_log_gradient(x_s, s, sample_theta)
    decision = policy(x_s, s).reshape(-1, 1)

    if gradient:
        grad_benefit = ((ips_weight/log_gradient_denominator) * decision * phi).sum(axis=0) / x_s.shape[0]
        return grad_benefit
    else:
        benefit = (ips_weight * decision).sum(axis=0) / x_s.shape[0]
        return benefit

def fairness_function(**fairness_kwargs):
    x = fairness_kwargs['x']
    s = fairness_kwargs['s']
    sample_theta = fairness_kwargs['sample_theta']
    policy = fairness_kwargs['policy']
    gradient = fairness_kwargs['gradient']

    pos_decision_idx = np.arange(s.shape[0]).reshape(-1, 1)

    s_0_idx = pos_decision_idx[s == 0]
    s_1_idx = pos_decision_idx[s == 1]

    return benefit_function(x[s_0_idx], s[s_0_idx], sample_theta, policy, gradient) - benefit_function(x[s_1_idx], s[s_1_idx], sample_theta, policy, gradient)


In [8]:
train(DIM_S, DIM_X,DIM_THETA, COST_FACTOR, NUM_DECISIONS, FRACTION_PROTECTED, LEARNING_RATE, FAIRNESS_RATE, BATCH_SIZE, NUM_ITERATIONS, T, fairness_function, IdentityFeatureMap(DIM_THETA))

: 0.07933640789136559
Time step 1544
Utility: 0.0700832266325224
Time step 1545
Utility: 0.07908287434353783
Time step 1546
Utility: 0.07928166107815318
Time step 1547
Utility: 0.08514315192158883
Time step 1548
Utility: 0.07646310432569972
Time step 1549
Utility: 0.06769220846233227
Time step 1550
Utility: 0.08285912999494181
Time step 1551
Utility: 0.07616560509554138
Time step 1552
Utility: 0.06558256821414714
Time step 1553
Utility: 0.07505128205128203
Time step 1554
Utility: 0.07084866167702648
Time step 1555
Utility: 0.07528794471461478
Time step 1556
Utility: 0.08122838478288713
Time step 1557
Utility: 0.07454545454545453
Time step 1558
Utility: 0.07161210382157296
Time step 1559
Utility: 0.06780605139891124
Time step 1560
Utility: 0.07362405823011109
Time step 1561
Utility: 0.07163170763093551
Time step 1562
Utility: 0.07660252325729577
Time step 1563
Utility: 0.07027371101209418
Time step 1564
Utility: 0.0747257053291536
Time step 1565
Utility: 0.07582040193335027
Time step 15